In [4]:
# Run me to hide code cells

from IPython.core.display import display, HTML
display(HTML(r"""<style id=hide>div.input{display:none;}</style>
<button type="button"onclick="var myStyle = document.getElementById('hide').sheet;
myStyle.insertRule('div.input{display:inherit !important;}', 0);">Show the code</button>"""))

In [5]:
import json
import os
import requests


'''
This python file defines some functions that are used regually in this project.
'''


def read_text_file(file_name: str) -> str:
    '''
    This function is used to read the text file and return the content.
    '''
    with open(file_name, 'r+', encoding="utf-8") as file:
        return file.read()


def read_json_file(file_name: str):
    '''
    This function is used to read the json file and return the content.
    '''
    return json.loads(read_text_file(file_name))


def download_text_file(url: str) -> str:
    '''
    This function is to use request to download the content into text format.
    '''
    return requests.get(url).text


def download_json_file(url: str):
    '''
    This function is use request to download the content into json format.
    '''
    return requests.get(url).json()


def write_file(file_name: str, content: str) -> str:
    '''
    This function is used to write the downloaded content in text or json format into files.
    '''
    # If the directory path does not exist, create one.
    os.makedirs(os.path.dirname(file_name), exist_ok=True)
    # If the file_name does exist, print 'the file has already been downloaded'.
    if os.path.isfile(file_name):
        print(f'{file_name} has already been downloaded')
    # Otherwise, write the content into files and print 'the file has just been downloaded.'
    else:
        with open(file_name, 'w+', encoding='utf-8') as file:
            print(f'{file_name} has just been downloaded.')
            file.write(content)

In [12]:
import json
from glob import glob
from utils import read_json_file
import pandas as pd
import sqlite3

'''
Since the aim of this project is to investigate the necessary job skills required for each position. 
The fact refers to each job details that is posted as well as the fact table is utilized for storing a collection of measures 
such as section id, line id, job details, etc. 
The Job Info Dimensional Table contains all the dimensions of each job such as the job title, job company, job area, etc.
The grain of fact table are job_id, section_id, and line_id which can be used to identify each job details. 
The grain of dimensional table is job_id which could be used to identify job information.

Therefore, this python file is used for transforming the data into Job Details Fact table and Job Info Dimension Table.
'''


def remove_key(d: dict) -> dict:
    '''
    This function is to remove the specific key from the dictionary.
    '''
    d_copy = d.copy()
    if d_copy.get('job_lines'):
        d_copy.pop('job_lines')
    return d_copy


def transfer_job_info(jobs) -> pd.DataFrame:
    '''
    This function is to remove the job_details from the data file and save the rest into job_info dataframe.
    '''
    job_info = [remove_key(job) for job in jobs]
    return pd.DataFrame(job_info)


def transfer_uls(jobs) -> pd.DataFrame:
    '''
    This function is to save job_details into dataframe.
    '''
    data = []
    for job in jobs:
        uls = job.get('job_lines', [])
        # Add section_index(section_id) and line_index(line_id) to every line of the job_details (uls)
        for ul_idx, ul in enumerate(uls):
            for li_idx, li in enumerate(ul):
                # Append the job_id, section_id, line_id and detail into the data list.
                data.append({
                    'job_id': job.get('job_id'),
                    'section_id': ul_idx + 1,
                    'line_id': li_idx + 1,
                    'detail': li
                })
    return pd.DataFrame(data)


job_locations = ['Melbourne', 'Sydney', 'Brisbane', 'Perth',
                'Adelaide', 'ACT', 'Hobart', 'Gold Coast']

job_details = []

Read all the json files into list job_details
for job_location in job_locations:
    for file in glob(f'downloads/seek/dataanalyst/{job_location}/*_details.json'):
        job_details.append(read_json_file(file))


Save the job_details including section_index, line_index to CSV file
details_df = transfer_uls(job_details)
details_df.to_csv('seek_data_analyst_job_details.csv', index=False)

# Save the job_info Dimension Table into CSV file including the rest information such as job title, job company, etc.
info_df = transfer_job_info(job_details)
info_df.to_csv('seek_data_analyst_job_info.csv', index=False)

# Conncet to the SQLite3 and tranfer the data into SQL files
con = sqlite3.connect('data_analyst_jobs_details.sqlite3')
details_df.to_sql('fact_job_details', con=con, index=False, if_exists='replace')

con = sqlite3.connect('data_analyst_jobs_info.sqlite3')
info_df.to_sql('dim_job_info', con=con, index=False, if_exists='replace')

In [ ]:
import pandas as pd
import re


'''
This python is used for job_salary data cleaning even though the objectives of this project 
do not include the job salary analysis.
'''

# Read the csv file into pandas dataframe.
job_details = pd.read_csv('seek_data_analyst_job_info.csv')

# job_salary data cleaning
# Create two empty lsit for the new columns. 
# salary_down means the lower limit of the salary from the job_info.
salary_down = []
# salary_up means the upper limit of the salary from the job_info.
salary_up = []

for i in job_details['job_salary']:
    # Use Regular Expression to extract the salary data and transform them into float numbers from the database.
    salary = re.findall(r'[\$-]([\d, kK]+)', str(i))
    salary_new1 = [s.replace(',', '') for s in salary]
    salary_new2 = [s.replace('k', '000') for s in salary_new1]
    salary_new3 = [s.replace('K', '000') for s in salary_new2]
    # Remove the blank
    salary_new4 = [s.replace(' ', '') for s in salary_new3]
    # Remove the null value
    salary_new5 = [s for s in salary_new4 if s != '']
    
    # Append the transformed data into the list.
    # If there is only one salary value, append it into the salary_down list.
    if len(salary_new5) == 1 and salary_new5[0].isdigit():
        salary_down.append(float(salary_new5[0]))
        salary_up.append('')
    # If there are two salary values, append them into the salary_down and salary_up list separately.
    elif len(salary_new5) == 2:
        salary_down.append(float(salary_new5[0]))
        salary_up.append(float(salary_new5[1]))
    # If there is no salary information, append null value into the lists.
    else:
        salary_down.append('')
        salary_up.append('')

# Create two new columns to the job_details dataframe.
job_details['job_salary_down'] = salary_down
job_details['job_salary_up'] = salary_up

# Based on different salary range, the job salary can be divided into following three categories: 
# If salary is lower than 200, it is paid by per hour;
# If salary is greater than 200 and lower than 2000, it is paid by per day;
# If salary is greater than 2000, it is paid by per annual.
# In such way, the annual salary can be calculated according to the full-time working hours from Australian government website. 
# There are 251 working days or 2008 working hours per year for a full-time working position.

job_details['annual_salary_down'] = job_details['job_salary_down']

job_details.annual_salary_down[(200 > job_details['job_salary_down'])] = job_details['job_salary_down'] * 2008
job_details.annual_salary_down[(200 < job_details['job_salary_down'])
                          & (job_details['job_salary_down'] < 2000)] = job_details['job_salary_down'] * 251


job_details['annual_salary_up'] = job_details['job_salary_up']

job_details.annual_salary_up[(200 > job_details['job_salary_down'])] = job_details['job_salary_up'] * 2008
job_details.annual_salary_up[(200 < job_details['job_salary_down'])
                          & (job_details['job_salary_down'] < 2000)] = job_details['job_salary_up'] * 251

# Finally, the data can be converted into a CSV file and needs to be manually checked for errors. 
job_details.to_csv('seek_data_analyst_job_info_cleaned.csv', index=False)


In [ ]:
from datetime import datetime, timedelta
import pandas as pd
import sqlite3


'''
This python file is used to transfrom datetime data into consistent data formats.

'''


def getdate(days_ago: int, hours_ago: int, minutes_ago: int):
    '''
    This function is used to transform the datetime to '%Y-%m-%d %H:%M:%S' data format.
    '''
    time_now = datetime.utcnow()
    time_diff = timedelta(days=days_ago, hours=hours_ago, minutes=minutes_ago)
    return (time_now - time_diff).strftime('%Y-%m-%d %H:%M:%S')


# Read the CSV file into pandas dataframe
job_details = pd.read_csv('seek_data_analyst_job_info_cleaned.csv')
# Create a new list for the cleaned and transformed job_posted_time
job_posted_time = []

'''
The job Ad posted time usually have the following suffixes:
1. 'm' means the job ad was posted minutes ago; 
2. 'h' means the job ad was posted hours before; 
3. 'd' means the job ad was posted days ago.
Based on these suffixes, we can use the Python datetime library 
to calculate the Universal Time Coordinated date of the job ad posted time.
'''
for dates in job_details['job_listing_date']:
    if 'd' in str(dates):
        days_ago = int(dates.split('d')[0])
        job_posted_time.append(getdate(days_ago, 0, 0))
    elif 'h' in str(dates):
        hours_ago = int(dates.split('h')[0])
        job_posted_time.append(getdate(0, hours_ago, 0))
    elif 'm' in str(dates):
        minutes_ago = int(dates.split('m')[0])
        job_posted_time.append(getdate(0, 0, minutes_ago))
    else:
        job_posted_time.append('')

# Create new column to the job_posted_time dataframe.
job_details['job_posted_time'] = job_posted_time

#### Filter the Related Recruitment Data for Data Analyst, Data Engineer, Data Scientist from More Than 10,000 Job Descriptions.

The search algorithm from SEEK website brings up lots of irrelevant job titles, such as business intelligence, accountant, etc. Therefore, before further analysis, we need to filter out the related recruitment data for Data Analyst, Data Engineer, Data Scientist. In this project, we use SQL LIKE and UNLIKE operator to identify whether if the job title belongs to these three titles or not.
Finally, from 16,000 job advertisements, there are 2995 job advertisements for Data Analyst, 1426 job advertisements for Data Engineer, and 729 job advertisements for Data Scientist. 

---

```sql
WITH 
enriched AS (
	SELECT job_id, job_title
			, CASE 
			WHEN job_title LIKE '%data%analyst%'  THEN 'data analyst'  
			WHEN job_title LIKE '%bi%'  THEN 'data analyst'  
			WHEN job_title LIKE '%tableau%'  THEN 'data analyst'
			WHEN job_title like '%sql%' THEN 'data analyst'		
			WHEN job_title LIKE '%analytic%'  THEN 'data analyst' 
			WHEN job_title LIKE '%data%modeller%'  THEN 'data analyst' 
			WHEN job_title LIKE '%visualisation%'  THEN 'data analyst' 
			WHEN job_title LIKE '%business%intelligence%'  THEN 'data analyst'  
			WHEN job_title like '%insight%' THEN 'data analyst'
			WHEN job_title like '%reporting%' THEN 'data analyst'
			WHEN job_title like '%model%' THEN 'data analyst'
			
			WHEN job_title like '%data%engineer%' THEN 'data engineer'
			WHEN job_title like '%warehouse%' THEN 'data engineer'
			WHEN job_title like '%architect%' THEN 'data engineer'
			WHEN job_title like '%snowflake%' THEN 'data engineer'
			WHEN job_title like '%etl%' THEN 'data engineer'
			WHEN job_title like '%api%' THEN 'data engineer'
			WHEN job_title like '%cloud%' THEN 'data engineer'
			WHEN job_title like '%aws%' THEN 'data engineer'
			WHEN job_title like '%kafka%' THEN 'data engineer'
			WHEN job_title like '%pipeline%' THEN 'data engineer'
			WHEN job_title like '%migration%' THEN 'data engineer'
			
			WHEN job_title like '%scientist%' THEN 'data scientist'
			WHEN job_title like '%science%' THEN 'data scientist'
			WHEN job_title like '%machine%learning%' THEN 'data scientist'
			WHEN job_title like '% ai%' THEN 'data scientist'
			WHEN job_title like '%ai %' THEN 'data scientist'
			WHEN job_title like '%ml%' THEN 'data scientist'
			WHEN job_title like '%computational%statistics%' then 'data scientist'
			WHEN job_title like '%quantitative%' then 'data scientist'
			WHEN job_title like '%artificial%intelligence%' then 'data scientist'
			
			WHEN job_title like '%business%analyst%' THEN 'else'
			WHEN job_title like '%accountant%' THEN 'else'
			WHEN job_title like '%sales%' THEN 'else'
			WHEN job_title like '%product%' THEN 'else'
			WHEN job_title like '%dev%ops%' THEN 'else'
			WHEN job_title like '%graduate%' THEN 'else'
			WHEN job_title LIKE '%developer%' THEN 'else'
			WHEN job_title LIKE '%functional%' THEN 'else'
			WHEN job_title LIKE '%financial%' THEN 'else'
			WHEN job_title LIKE '%finance%' THEN 'else'
			WHEN job_title LIKE '%manager%' THEN 'else'
			WHEN job_title LIKE '%project%' THEN 'else'
			WHEN job_title LIKE '%program%' THEN 'else'
			
			END AS job_standard_name
	FROM dim_job_info
	ORDER BY 2 DESC
	)

, filtered_job_details AS (
	SELECT *
	FROM dim_job_details
	WHERE job_id IN (
		SELECT job_id FROM enriched 
 		WHERE job_standard_name = 'data analyst' )
-- 		WHERE job_standard_name = 'data engineer' )
--		WHERE job_standard_name = 'data scientist' )
	)

-- Select the job_details of each job_standard_name and transfrom them into CSV files

SELECT *
FROM filtered_job_details

-- Count the JD numbers of each job_standard_name
SELECT job_standard_name, count(1)
FROM enriched
GROUP BY 1

```

---

In [ ]:
import re
import pandas as pd
from typing import Tuple, List, Iterable, Any
import nltk
from nltk.corpus import stopwords


'''
From the filtered JDs above, this python file is used to calculate the appeared frequency of each word and phrase. 
Then we can filter out the meaningful word and phrase with high appeared frequency including technical and soft skills.
'''

# Download the stopword from nltk library.
# nltk.download('stopwords')


def ngram(l: Iterable[Any], gram: int) -> List[Any]:
    '''
    Return the phrases with length of gram from the list.
    '''
    l = list(l)
    return [l[i: i + gram] for i in range(len(l) - gram + 1)]


def get_words(line: str, gram: int) -> List[str]:
    '''
    This function is to return words or phrases with defined length.    
    '''
    words = re.findall(r'[a-zA-Z]+', line.lower())
    # If gram equals to one, filter out the stop_words and return the other words.
    if gram == 1:
        stop_words = stopwords.words('english')
        words = [word for word in words if word not in stop_words]
    # Otherwise, return the Return the phrases with length of gram from the sentence.
    else:
        words = [' '.join(word) for word in ngram(words, gram)]
    return words


def word_count(stats: dict, sentence: str, gram: int) -> dict:
    '''
    Return the calculated appeared frequency of each word and phrase from each line of the job_detail. 
    '''
    if isinstance(sentence, str):
        for word in get_words(sentence, gram):
            # If the word already exisits in the dict, the value pluses one.
            if word in stats:
                stats[word] += 1
            # Otherwise, append the word with value one to the dict.
            else:
                stats[word] = 1
    return stats
    

def sorted_word_count(d: dict) -> Tuple:
    '''
    This function is used to sort the word count dictionary with values in reverse order.
    '''
    return sorted(list(d.items()), key=lambda x: x[-1], reverse=True)


# read the job details file
wc = {}
job_uls = pd.read_csv('seek_data_analyst_job_details.csv')

# For each line of the job_details, count the appeared frequency of the word or phrases.
for ul in job_uls['detail']:
    wc = word_count(wc, ul, gram=1)

# Sort the word count dictionary 'wc' with values in reverse order.
for i in sorted_word_count(wc):
    print(i)

# Transform the sorted result and save it into CSV file for further analysis to filter out the meaningful words.
df = pd.DataFrame(sorted_word_count(wc))
df.to_csv('seek_data_scientist_gram_1.csv', index=False)

In [ ]:
import re
import pandas as pd
from typing import Tuple, List, Iterable, Any
import nltk
from nltk.corpus import stopwords


'''
After the analysis above, we filter out 46 highly-demand technical and 12 soft skills in the job market 
as shown in the keywords list.
Then we can use Power BI to draw the word cloud and caculated the averaged line_id of each skill.
'''


def filtered_keywords(sentence: str) -> list:
    '''
    This function is to filter out the skills from each sentence.
    '''
    result = []
    if isinstance(sentence, str):
        for word in keywords:
            if word.lower() in sentence.lower():
                result.append(word)
    return result


def ngram(l: Iterable[Any], gram: int) -> List[Any]:
    '''
    Return the phrases with length of gram from the list.
    '''
    l = list(l)
    return [l[i: i + gram] for i in range(len(l) - gram + 1)]
    
    
def get_words(line: str, gram: int) -> List[str]:
    '''
    This function is to return words or phrases with defined length.    
    '''
    words = re.findall(r'[a-zA-Z]+', line.lower())
    # If gram equals to one, filter out the stop_words and return the other words.
    if gram == 1:
        stop_words = stopwords.words('english')
        words = [word for word in words if word not in stop_words]
    # Otherwise, return the Return the phrases with length of gram from the sentence.
    else:
        words = [' '.join(word) for word in ngram(words, gram)]
    return words

    
keywords = ['SQL', 'Excel', 'PowerBI', 'Tableau', 'Python', 'Cloud'
        , 'Azure', 'AWS', 'GCP', 'API', 'Pipeline', 'Dimension Modelling', 'ETL', 'ELT'
        , 'DevOps', 'CI CD', 'Spark', 'Java', 'Scala', 'Oracle', 'Kubernetes', 'Docker'
        , 'Apache', 'Kafka', 'Linux', 'Snowflake', 'Data Warehouse'
        , 'Data Modelling', 'Data Visualisation', 'Data Migration', 'Data Management'
        , 'Data Integration', 'Data Platform', 'Data Architecture', 'Data Factory', 'Databricks', 'Data Science'
        , 'Machine Learning', 'Computer Science', 'Research', 'Statistic', 'Mathematics'
        , 'Quantitative', 'Algorithm', 'Deep Learning', 'Statistical Analysis'
        , 'Communication Skill', 'Stakeholder', 'Reporting', 'Agile'
        , 'Project Management', 'Business Intelligence', 'Decision Making', 'Interpersonal'
        , 'Time Management', 'Troubleshoot', 'Tertiary Qualification', 'PhD']
    

# read the job details file
exsited_keyword = []
job_uls = pd.read_csv('seek_data_analyst_job_details.csv)

# filter out 46 highly-demand technical and 12 soft skills in the job market as shown in the keywords list.                      
for ul in job_uls['detail']:
    exsited_keyword.append(filtered_keywords(ul))

job_uls['exsited_keyword'] = exsited_keyword
# Save the result into CSV file. Then we can use Power BI to draw the word cloud.                      
job_uls.to_csv('seek_data_analyst_filtered_keywords.csv', index=False)                  
                      
# Create the dictionary to caculate the averaged line_id of each skill.
keywords_dict = {'SQL':[], 'Excel':[], 'Power BI':[], 'Tableau':[], 'Python':[], 'Cloud':[], 'Azure':[]
        , 'AWS':[], 'GCP':[], 'API':[], 'Pipeline':[], 'Dimension Modelling':[], 'ETL':[], 'ELT':[]
        , 'DevOps':[], 'CI CD':[], 'Spark':[], 'Java':[], 'Scala':[], 'Oracle': [], 'Kubernetes': []
        , 'Docker':[], 'Apache': [], 'Kafka':[], 'Linux':[], 'Snowflake':[], 'Data Warehouse':[]
        , 'Data Modelling':[], 'Data Visualisation':[], 'Data Migration':[], 'Data Management':[]
        , 'Data Integration':[], 'Data Platform':[], 'Data Architecture':[], 'Data Factory':[], 'Data Science':[]
        , 'Machine Learning':[], 'Computer Science':[], 'Research':[], 'Statistic':[], 'Mathematics':[]
        , 'Quantitative':[], 'Algorithm':[], 'Deep Learning':[], 'Statistical Analysis':[]
        , 'Communication Skill':[], 'Stakeholder': [], 'Reporting':[], 'Agile':[]
        , 'Project Management':[], 'Business Intelligence':[], 'Decision Making':[], 'Interpersonal':[]
        , 'Time Management':[], 'Troubleshoot':[], 'Tertiary Qualification':[], 'PhD':[]}
                                          
for index, row in job_uls.iterrows():
    if isinstance(row['detail'], str):
        for word in get_words(row['detail'], 1):
            for skill in keywords:
                if word == skill.lower():
                    keywords_dict[skill].append(row['line_id'])
        for word in get_words(row['detail'], 2):
            for skill in keywords:
                if word == skill.lower():
                    keywords_dict[skill].append(row['line_id'])

skill_ranked_line = {}
                      
for key, value in keywords_dict.items():
    if len(value) != 0:
        skill_ranked_line[key] = sum(value) / len(value)

skill_ranked_line.to_csv('skill_ranked_line.csv', index=False)

#### Documentary Frequency Analysis and Skills Required to be a Data Analyst, Data Engineer and Data Scientist.

After we filter out the important skills, we can use SQL LIKE operator to identify which statements contain these keywords and count the distinct total number of job ads.

---

```sql
WITH 
enriched AS (
	SELECT job_id, job_title
			, CASE 
			WHEN job_title LIKE '%data%analyst%'  THEN 'data analyst'  
			WHEN job_title LIKE '%bi%'  THEN 'data analyst'  
			WHEN job_title LIKE '%tableau%'  THEN 'data analyst'
			WHEN job_title like '%sql%' THEN 'data analyst'		
			WHEN job_title LIKE '%analytic%'  THEN 'data analyst' 
			WHEN job_title LIKE '%data%modeller%'  THEN 'data analyst' 
			WHEN job_title LIKE '%visualisation%'  THEN 'data analyst' 
			WHEN job_title LIKE '%business%intelligence%'  THEN 'data analyst'  
			WHEN job_title like '%insight%' THEN 'data analyst'
			WHEN job_title like '%reporting%' THEN 'data analyst'
			WHEN job_title like '%model%' THEN 'data analyst'
			
			WHEN job_title like '%data%engineer%' THEN 'data engineer'
			WHEN job_title like '%warehouse%' THEN 'data engineer'
			WHEN job_title like '%architect%' THEN 'data engineer'
			WHEN job_title like '%snowflake%' THEN 'data engineer'
			WHEN job_title like '%etl%' THEN 'data engineer'
			WHEN job_title like '%api%' THEN 'data engineer'
			WHEN job_title like '%cloud%' THEN 'data engineer'
			WHEN job_title like '%aws%' THEN 'data engineer'
			WHEN job_title like '%kafka%' THEN 'data engineer'
			WHEN job_title like '%pipeline%' THEN 'data engineer'
			WHEN job_title like '%migration%' THEN 'data engineer'
			
			WHEN job_title like '%scientist%' THEN 'data scientist'
			WHEN job_title like '%science%' THEN 'data scientist'
			WHEN job_title like '%machine%learning%' THEN 'data scientist'
			WHEN job_title like '% ai%' THEN 'data scientist'
			WHEN job_title like '%ai %' THEN 'data scientist'
			WHEN job_title like '%ml%' THEN 'data scientist'
			WHEN job_title like '%computational%statistics%' then 'data scientist'
			WHEN job_title like '%quantitative%' then 'data scientist'
			WHEN job_title like '%artificial%intelligence%' then 'data scientist'
			
			WHEN job_title like '%business%analyst%' THEN 'else'
			WHEN job_title like '%accountant%' THEN 'else'
			WHEN job_title like '%sales%' THEN 'else'
			WHEN job_title like '%product%' THEN 'else'
			WHEN job_title like '%dev%ops%' THEN 'else'
			WHEN job_title like '%graduate%' THEN 'else'
			WHEN job_title LIKE '%developer%' THEN 'else'
			WHEN job_title LIKE '%functional%' THEN 'else'
			WHEN job_title LIKE '%financial%' THEN 'else'
			WHEN job_title LIKE '%finance%' THEN 'else'
			WHEN job_title LIKE '%manager%' THEN 'else'
			WHEN job_title LIKE '%project%' THEN 'else'
			WHEN job_title LIKE '%program%' THEN 'else'
			
			END AS job_standard_name
	FROM dim_job_info
	ORDER BY 2 DESC
	)

, filtered_job_details AS (
	SELECT *
	FROM dim_job_details
	WHERE job_id IN (
		SELECT job_id FROM enriched 
 		WHERE job_standard_name = 'data analyst' )
-- 		WHERE job_standard_name = 'data engineer' )
--		WHERE job_standard_name = 'data scientist' )
	)

, enriched_details AS (
	SELECT *
		, CASE WHEN detail LIKE '%sql%' THEN '1' END AS sql_counter
		, CASE WHEN detail LIKE '%excel%' THEN '1' END AS excel_counter
		, CASE WHEN detail LIKE '%power%bi%' THEN '1' END AS powerbi_counter
		, CASE WHEN detail LIKE '%tableau%' THEN '1' END AS tableau_counter
		, CASE WHEN detail LIKE '%python%' THEN '1' END AS python_counter
		, CASE WHEN detail LIKE '%cloud%' THEN '1' END AS cloud_counter
		, CASE WHEN detail LIKE '%azure%' THEN '1' END AS azure_counter
		, CASE WHEN detail LIKE '%aws%' THEN '1' END AS aws_counter
		, CASE WHEN detail LIKE '%gcp%' THEN '1' END AS gcp_counter
		
		, CASE WHEN detail LIKE '%api%' THEN '1' END AS api_counter
		, CASE WHEN detail LIKE '%pipeline%' THEN '1' END AS pipeline_counter
		, CASE WHEN detail LIKE '%dimension%' THEN '1' END AS dimension_counter
		, CASE WHEN detail LIKE '%etl%' THEN '1' END AS etl_counter
		, CASE WHEN detail LIKE '%elt%' THEN '1' END AS elt_counter
		, CASE WHEN detail LIKE '%devops%' THEN '1' END AS devops_counter
		, CASE WHEN detail LIKE '%ci%cd%' THEN '1' END AS cicd_counter
		, CASE WHEN detail LIKE '%spark%' THEN '1' END AS spark_counter
		, CASE WHEN detail LIKE '%java%' THEN '1' END AS java_counter
		, CASE WHEN detail LIKE '%scala%' THEN '1' END AS scala_counter
		, CASE WHEN detail LIKE '%kafka%' THEN '1' END AS kafka_counter
		, CASE WHEN detail LIKE '%linux%' THEN '1' END AS linux_counter
		, CASE WHEN detail LIKE '%snowflake%' THEN '1' END AS snowflake_counter	
		, CASE WHEN detail LIKE '%oracle%' THEN '1' END AS oracle_counter
		, CASE WHEN detail LIKE '%kubernetes%' THEN '1' END AS kubernetes_counter
		, CASE WHEN detail LIKE '%docker%' THEN '1' END AS docker_counter
		, CASE WHEN detail LIKE '%apache%' THEN '1' END AS apache_counter
		
		, CASE WHEN detail LIKE '%data%warehous%' THEN '1' END AS DW_counter
		, CASE WHEN detail LIKE '%data%modelling%' THEN '1' END AS DM_counter
		, CASE WHEN detail LIKE '%data%visualisation%' THEN '1' END AS DV_counter
		, CASE WHEN detail LIKE '%data%migration%' THEN '1' END AS DMI_counter
		, CASE WHEN detail LIKE '%data%management%' THEN '1' END AS DMA_counter
		, CASE WHEN detail LIKE '%data%integration%' THEN '1' END AS DI_counter
		, CASE WHEN detail LIKE '%data%platform%' THEN '1' END AS DP_counter
		, CASE WHEN detail LIKE '%data%architecture%' THEN '1' END AS DA_counter
		, CASE WHEN detail LIKE '%data%factory%' THEN '1' END AS DF_counter
		, CASE WHEN detail LIKE '%databricks%' THEN '1' END AS DB_counter
		
		, CASE WHEN detail LIKE '%data%science%' THEN '1' END AS DS_counter
		, CASE WHEN detail LIKE '%machine%learning%' THEN '1' END AS ML_counter
		, CASE WHEN detail LIKE '% ai %' THEN '1' END AS AI_counter
		, CASE WHEN detail LIKE '%computer%science%' THEN '1' END AS CS_counter
		, CASE WHEN detail LIKE '%research%' THEN '1' END AS research_counter
		, CASE WHEN detail LIKE '%statistic%' THEN '1' END AS statistic_counter
		, CASE WHEN detail LIKE '%mathematics%' THEN '1' END AS mathematics_counter
		, CASE WHEN detail LIKE '%quantitative%' THEN '1' END AS quantitative_counter
		, CASE WHEN detail LIKE '%algorithm%' THEN '1' END AS algorithm_counter
		, CASE WHEN detail LIKE '%deep%learning%' THEN '1' END AS DL_counter
		, CASE WHEN detail LIKE '%statistical%analysis%' THEN '1' END AS SA_counter
        
        , CASE WHEN detail LIKE '%communication%' THEN '1' END AS communication_counter
		, CASE WHEN detail LIKE '%reporting%' THEN '1' END AS reporting_counter
		, CASE WHEN detail LIKE '%agile%' THEN '1' END AS agile_counter
		
		, CASE WHEN detail LIKE '%stakeholder%' THEN '1' END AS stakeholder_counter
		, CASE WHEN detail LIKE '%project%management%' THEN '1' END AS PM_counter
		, CASE WHEN detail LIKE '%decision%making%' THEN '1' END AS DM_counter
		, CASE WHEN detail LIKE '%interpersonal%skills%' THEN '1' END AS IS_counter
		, CASE WHEN detail LIKE '%time%management%' THEN '1' END AS TM_counter
		, CASE WHEN detail LIKE '%troubleshoot%' THEN '1' END AS troubleshoot_counter
		, CASE WHEN detail LIKE '%tertiary%qualification%' THEN '1' END AS TQ_counter
		, CASE WHEN detail LIKE '%phd%' THEN '1' END AS phd_counter
		, CASE WHEN detail LIKE '%business%intelligence%' THEN '1' END AS BI_counter

	FROM filtered_job_details
	)
, job_details AS (
	SELECT job_id
		, SUM(sql_counter) > 0 AS has_sql
		, SUM(excel_counter) > 0 AS has_excel
		, SUM(powerbi_counter) > 0 AS has_powerbi
		, SUM(tableau_counter) > 0 AS has_tableau
		, SUM(python_counter) > 0 AS has_python
		, SUM(cloud_counter) > 0 AS has_cloud
		, SUM(azure_counter) > 0 AS has_azure
		, SUM(aws_counter) > 0 AS has_aws
		, SUM(gcp_counter) > 0 AS has_gcp
		
		, SUM(api_counter) > 0 AS has_api
		, SUM(pipeline_counter) > 0 AS has_pipeline
		, SUM(dimension_counter) > 0 AS has_dimension
		, SUM(etl_counter) > 0 AS has_etl
		, SUM(elt_counter) > 0 AS has_elt
		, SUM(devops_counter) > 0 AS has_devops
		, SUM(cicd_counter) > 0 AS has_cicd
		, SUM(spark_counter) > 0 AS has_spark
		, SUM(java_counter) > 0 AS has_java
		, SUM(scala_counter) > 0 AS has_scala
		, SUM(kafka_counter) > 0 AS has_kafka
		, SUM(linux_counter) > 0 AS has_linux
		, SUM(snowflake_counter) > 0 AS has_snowflake
		, SUM(oracle_counter) > 0 AS has_oracle
		, SUM(kubernetes_counter) > 0 AS has_kubernetes
		, SUM(docker_counter) > 0 AS has_docker
		, SUM(apache_counter) > 0 AS has_apache

		, SUM(DW_counter) > 0 AS has_DW
		, SUM(DM_counter) > 0 AS has_DM
		, SUM(DV_counter) > 0 AS has_DV
		, SUM(DMI_counter) > 0 AS has_DMI
		, SUM(DMA_counter) > 0 AS has_DMA
		, SUM(DI_counter) > 0 AS has_DI
		, SUM(DP_counter) > 0 AS has_DP
		, SUM(DA_counter) > 0 AS has_DA
		, SUM(DF_counter) > 0 AS has_DF
		, SUM(DB_counter) > 0 AS has_DB
		
		, SUM(DS_counter) > 0 AS has_DS
		, SUM(ML_counter) > 0 AS has_ML
		, SUM(AI_counter) > 0 AS has_AI
		, SUM(CS_counter) > 0 AS has_CS
		, SUM(research_counter) > 0 AS has_research
		, SUM(statistic_counter) > 0 AS has_statistic
		, SUM(mathematics_counter) > 0 AS has_mathematics
		, SUM(quantitative_counter) > 0 AS has_quantitative
		, SUM(algorithm_counter) > 0 AS has_algorithm
		, SUM(DM_counter) > 0 AS has_DM
		, SUM(SA_counter) > 0 AS has_SA
        
        , SUM(communication_counter) > 0 AS has_communication
		, SUM(reporting_counter) > 0 AS has_reporting
		, SUM(agile_counter) > 0 AS has_agile
		, SUM(stakeholder_counter) > 0 AS has_stakeholder
		, SUM(PM_counter) > 0 AS has_PM
		, SUM(DM_counter) > 0 AS has_DM
		, SUM(IS_counter) > 0 AS has_IS
		, SUM(TM_counter) > 0 AS has_TM
		, SUM(troubleshoot_counter) > 0 AS has_troubleshoot
		, SUM(TQ_counter) > 0 AS has_TQ
		, SUM(phd_counter) > 0 AS has_phd
		, SUM(BI_counter) > 0 AS has_BI

	FROM enriched_details
	GROUP BY job_id
	)

SELECT 
(SELECT COUNT(DISTINCT job_id) FROM filtered_job_details) AS total_jobs
	, SUM(has_sql) AS sql_jobs
	, SUM(has_excel) AS excel_jobs
	, SUM(has_powerbi) AS powerbi_jobs
	, SUM(has_tableau) AS tableau_jobs
	, SUM(has_python) AS python_jobs
	, SUM(has_cloud) AS cloud_jobs
	, SUM(has_azure) AS azure_jobs
	, SUM(has_aws) AS aws_jobs
	, SUM(has_gcp) AS gcp_jobs
	
	, SUM(has_api) AS api_jobs
	, SUM(has_pipeline) AS pipeline_jobs
	, SUM(has_dimension) AS dimension_jobs
	, SUM(has_etl) AS etl_jobs
	, SUM(has_elt) AS elt_jobs
	, SUM(has_devops) AS devops_jobs
	, SUM(has_cicd) AS cicd_jobs
	, SUM(has_spark) AS spark_jobs
	, SUM(has_java) AS java_jobs
	, SUM(has_scala) AS scala_jobs
	, SUM(has_kafka) AS kafka_jobs
	, SUM(has_linux) AS linux_jobs
	, SUM(has_snowflake) AS snowflake_jobs
	, SUM(has_oracle) AS oracle_jobs
	, SUM(has_kubernetes) AS kubernetes_jobs
	, SUM(has_docker) AS docker_jobs
	, SUM(has_apache) AS apache_jobs

	, SUM(has_DW) AS datawarehouse_jobs
	, SUM(has_DM) AS datamodelling_jobs
	, SUM(has_DV) AS datavisualisation_jobs
	, SUM(has_DMI) AS datamigration_jobs
	, SUM(has_DMA) AS datamanagement_jobs
	, SUM(has_DI) AS dataintegration_jobs
	, SUM(has_DP) AS dataplatform_jobs
	, SUM(has_DA) AS dataarchitecture_jobs
	, SUM(has_DF) AS datafactory_jobs
	, SUM(has_DB) AS databricks_jobs
	
	, SUM(has_DS) AS datascience_jobs
	, SUM(has_ML) AS machinelearning_jobs
	, SUM(has_AI) AS Artificialintelligence_jobs
	, SUM(has_CS) AS computerscience_jobs
	, SUM(has_research) AS research_jobs
	, SUM(has_statistic) AS statistic_jobs
	, SUM(has_mathematics) AS mathematics_jobs
	, SUM(has_quantitative) AS quantitative_jobs
	, SUM(has_algorithm) AS algorithm_jobs
	, SUM(has_DM) AS deeplearning_jobs
	, SUM(has_SA) AS statisticalanalysis_jobs
    
    , SUM(has_communication) AS Communication_Skill
	, SUM(has_reporting) AS Reporting
	, SUM(has_agile) AS Agile
	, SUM(has_stakeholder) AS Stakeholder
	, SUM(has_PM) AS Project_Management
	, SUM(has_BI) AS Business_Intelligence
	, SUM(has_DM) AS Decision_Making
	, SUM(has_IS) AS Interpersonal_Skill
	, SUM(has_TM) AS Time_Management
	, SUM(has_troubleshoot) AS Troubleshooting
	, SUM(has_TQ) AS Tertiary_Qualification
	, SUM(has_phd) AS PhD
    
	
FROM job_details


```

---

**Ranked Line in the Job Description of the Skills**

In order to illustrate further, we assume that people always would like to put the most important information first, for example, the higher the position in the job advertisement. Then we analyze the average appeared line number of these important skills. We categorize their first occurrence line numbers into three range: 1~3, 4~5 and 6+,

---

```sql

WITH 
enriched AS (
	SELECT job_id, job_title
			, CASE 
			WHEN job_title LIKE '%data%analyst%'  THEN 'data analyst'  
			WHEN job_title LIKE '%bi%'  THEN 'data analyst'  
			WHEN job_title LIKE '%tableau%'  THEN 'data analyst'
			WHEN job_title like '%sql%' THEN 'data analyst'		
			WHEN job_title LIKE '%analytic%'  THEN 'data analyst' 
			WHEN job_title LIKE '%data%modeller%'  THEN 'data analyst' 
			WHEN job_title LIKE '%visualisation%'  THEN 'data analyst' 
			WHEN job_title LIKE '%business%intelligence%'  THEN 'data analyst'  
			WHEN job_title like '%insight%' THEN 'data analyst'
			WHEN job_title like '%reporting%' THEN 'data analyst'
			WHEN job_title like '%model%' THEN 'data analyst'
			
			WHEN job_title like '%data%engineer%' THEN 'data engineer'
			WHEN job_title like '%warehouse%' THEN 'data engineer'
			WHEN job_title like '%architect%' THEN 'data engineer'
			WHEN job_title like '%snowflake%' THEN 'data engineer'
			WHEN job_title like '%etl%' THEN 'data engineer'
			WHEN job_title like '%api%' THEN 'data engineer'
			WHEN job_title like '%cloud%' THEN 'data engineer'
			WHEN job_title like '%aws%' THEN 'data engineer'
			WHEN job_title like '%kafka%' THEN 'data engineer'
			WHEN job_title like '%pipeline%' THEN 'data engineer'
			WHEN job_title like '%migration%' THEN 'data engineer'
			
			WHEN job_title like '%scientist%' THEN 'data scientist'
			WHEN job_title like '%science%' THEN 'data scientist'
			WHEN job_title like '%machine%learning%' THEN 'data scientist'
			WHEN job_title like '% ai%' THEN 'data scientist'
			WHEN job_title like '%ai %' THEN 'data scientist'
			WHEN job_title like '%ml%' THEN 'data scientist'
			WHEN job_title like '%computational%statistics%' then 'data scientist'
			WHEN job_title like '%quantitative%' then 'data scientist'
			WHEN job_title like '%artificial%intelligence%' then 'data scientist'
			END AS job_standard_name
            
	FROM dim_job_info
	ORDER BY 2 DESC
	)

, filtered_job_details AS (
	SELECT *
	FROM dim_job_details
	WHERE job_id IN (
		SELECT job_id FROM enriched 
-- 		WHERE job_standard_name = 'data analyst' )
-- 		WHERE job_standard_name = 'data engineer' )
		WHERE job_standard_name = 'data scientist' )
)
, filtered_skills AS (

	SELECT job_id, 'SQL' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%sql%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'Excel' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%excel%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'PowerBI' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%power%bi%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'Tableau' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%tableau%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'Python' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%python%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'Cloud' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%cloud%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'Azure' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%azure%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'AWS' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%aws%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'GCP' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%gcp%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'API' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%api%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'Pipeline' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%pipeline%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'Dimension Modelling' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%dimension%modelling%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'ETL' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%etl%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'ELT' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%elt%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'DevOps' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%devops%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'CI CD' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%ci%cd%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'Spark' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%spark%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'Java' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%java%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'Scala' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%scala%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'Oracle' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%oracle%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'Kubernetes' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%kubernetes%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'Docker' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%docker%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'Apache' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%apache%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'Kafka' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%kafka%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'Linux' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%linux%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'Snowflake' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%snowflake%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'Data Warehouse' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%data%warehouse%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'Data Modelling' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%data%modelling%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'Data Visualisation' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%data%visualisation%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'Data Migration' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%data%migration%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'Data Management' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%data%management%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'Data Integration' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%data%integration%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'Data Platform' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%data%platform%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'Data Architecture' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%data%architecture%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'Data Factory' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%data%factory%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'Databricks' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%databricks%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'Data Science' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%data%science%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'Machine Learning' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%machine%learning%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'Computer Science' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%computer%science%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'Research' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%research%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'Statistic' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%statistic%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'Mathematics' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%mathematics%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'Quantitative' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%quantitative%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'Algorithm' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%algorithm%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'Deep Learning' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%deep%learning%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'Statistical Analysis' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%statistical%analysis%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'Communication Skill' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%communication%skill%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'Stakeholder' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%stakeholder%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'Reporting' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%reporting%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'Agile' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%agile%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'Project Management' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%project%management%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'Business Intelligence' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%business%intelligence%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'Decision Making' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%decision%making%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'Interpersonal' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%interpersonal%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'Time Management' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%time%management%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'Troubleshoot' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%troubleshoot%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'Tertiary Qualification' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%tertiary%qualification%'
	GROUP BY job_id

	UNION
	SELECT job_id, 'PhD' AS skill
	, MIN(line_id) AS line_id
	FROM filtered_job_details
	WHERE detail LIKE '%phd%'
	GROUP BY job_id

)
SELECT *
	, CASE 
		WHEN line_id <= 3 THEN '1~3'
		WHEN (line_id > 3 AND line_id <= 5) THEN '4~5'
		ELSE '6+' END AS line_id_category
FROM filtered_skills

```

---